# Ant Colony Algorithm

In [2]:
import numpy as np

class AntColonyOptimizer:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        Ant colony optimizer for the traveling salesman problem.
        """
        self.distances = np.clip(distances, 1e-10, None)  # Avoid division by zero in distances.
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        shortest_dist = float('inf')
        for _ in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path, shortest_dist = self.find_shortest_path(all_paths)
        return shortest_path, shortest_dist

    def gen_path_dist(self, path):
        return sum([self.distances[path[i], path[i+1]] for i in range(len(path)-1)])

    def gen_all_paths(self):
        all_paths = []
        for _ in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = [start]
        visited = set(path)
        while len(path) < len(self.distances):
            move = self.pick_move(self.pheromone[path[-1]], self.distances[path[-1]], visited)
            path.append(move)
            visited.add(move)
        path.append(start)  # returning to the start
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)
        if np.sum(row) == 0:
            row = np.ones_like(row)  # Avoid division by zero in probabilities.
        norm_row = row / row.sum()
        move = np.random.choice(self.all_inds, 1, p=norm_row)[0]
        return move

    def spread_pheromone(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in range(len(path)-1):
                self.pheromone[path[move], path[move+1]] += 1.0 / self.distances[path[move], path[move+1]]
        self.pheromone *= self.decay

    def find_shortest_path(self, all_paths):
        shortest_path = min(all_paths, key=lambda x: x[1])
        return shortest_path[0], shortest_path[1]

# Example usage
distances = np.random.rand(10, 10)
distances = (distances + distances.T) / 2  # make symmetric
np.fill_diagonal(distances, 0)
aco = AntColonyOptimizer(distances, n_ants=20, n_best=5, n_iterations=100, decay=0.95, alpha=1, beta=2)
path, dist = aco.run()
print("Shortest path: ", path)
print("Shortest distance: ", dist)

Shortest path:  [0, np.int64(3), np.int64(5), np.int64(2), np.int64(9), np.int64(7), np.int64(8), np.int64(1), np.int64(6), np.int64(4), 0]
Shortest distance:  2.4694807485556134
